<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/11_all_getty_skos_matches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIY

Sometimes the API doesn't exactly provide you with a filter or search parameter to do what you want. In those cases you might need to make more of an effort to get the results that you want, but because the data is all available, there's nothing you can't do with a little script.

For this script we'll work with the [Onroerend Erfgoed Thesaurus](https://thesaurus.onroerenderfgoed.be). We'll use the Conceptscheme of `Erfgoedtypes` to see how many of them are linked to the Getty AAT thesaurus.

The thesaurus API supports checking which concepts are linked to a certain AAT concept, but it does not support searching for all concepts that are linked to any AAT concept.

In [ ]:
import requests

THESAURUS_HOST = 'https://thesaurus.onroerenderfgoed.be'
ERFGOEDTYPES_URL = THESAURUS_HOST + '/conceptschemes/ERFGOEDTYPES/c'

session = requests.Session()

session.headers.update({'Accept': 'application/json'})

res = session.get(
    ERFGOEDTYPES_URL,
    params={
        'match': 'http://vocab.getty.edu/aat/300005241'
    }
)

print(res.json())

The previous example works well, but we would like to see all concepts in this thesaurus that have a link to the AAT.

In [ ]:
import requests

THESAURUS_HOST = 'https://thesaurus.onroerenderfgoed.be'
ERFGOEDTYPES_URL = THESAURUS_HOST + '/conceptschemes/ERFGOEDTYPES/c'

session = requests.Session()

session.headers.update({'Accept': 'application/json'})

# Get all erfgoedtypes
# This collection does not support pagination and always sends everything
res = session.get(
    ERFGOEDTYPES_URL,
    params = {
        'type': 'concept'
    }
)

# Make sure everything went well
res.raise_for_status()

data = res.json()

for concept in data:
  conceptresponse = session.get(concept['uri'])
  conceptresponse.raise_for_status()
  conceptdetail = conceptresponse.json()
  # Only look at concepts that have any matches at all
  if conceptdetail['matches']:
    print(conceptdetail['uri'])
    for matchtype, matchvalue in conceptdetail['matches'].items():
      print(f"{matchtype} {matchvalue}")